## Kernel: Python 3.8

In [ ]:
from dotenv import dotenv_values
from datetime import timedelta, datetime

import pandas as pd
import requests
import locale

## Get Authority

In [ ]:
config = dotenv_values(".env") # get values from .env file
locale.setlocale(locale.LC_ALL, "id") # set local language

def login() -> str:
    try:
        url = 'https://api.kampusmerdeka.kemdikbud.go.id/user/auth/login/mbkm'
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
        }
        account = {
            'email': config.get('EMAIL'),
            'password': config.get('PASSWORD')
        }
        response = requests.post(url, json=account, headers=headers)
        token = response.json()['data'].get('access_token')
    
    except Exception:
        raise ValueError(response.json()["error"]["message"])
    
    return token, account

(token, account) = login()
headers = dict()
headers["Accept"] = 'application/json'
headers["Authorization"] = f'Bearer {token}'
headers["User-Agent"] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'

print(f'login to {account["email"]} success')

## 1. Fetch ALL Mitra (Raw & Test)

In [ ]:
url = f'https://api.kampusmerdeka.kemdikbud.go.id/magang/browse/position?offset=0&limit=20&location_key=&mitra_key=&keyword=&sector_id='
response = requests.get(url, headers=headers)
reports = response.json().get('data')

print(reports)

## 2. Process Data FULL

In [58]:
url = f'https://api.kampusmerdeka.kemdikbud.go.id/magang/browse/position?offset=0&limit=10000&location_key=&mitra_key=&keyword=&sector_id='
response = requests.get(url, headers=headers)
mitra_data = response.json().get('data')

#===== List Initialization
id_position_cols = []
position_name_cols = []
mitra_name_cols = []
activity_name_cols = []
position_quota_cols = []
location_cols = []
activity_type_cols = []
logo_mitra_cols = []

# TODO: EXTRA FIELDS
start_duration_cols = []
end_duration_cols = []
start_registration_cols = []
end_registration_cols = []
credits_count_cols = []
location_code_cols = []
mitra_id_cols = []
certified_cols = []


for mitra_data_detail in mitra_data:
    #===== fetch data from mitra_data_detail dict in mitra_data list
    id_position = mitra_data_detail["id"]
    position_name = mitra_data_detail["name"]
    mitra_name = mitra_data_detail["mitra_name"]
    activity_name = mitra_data_detail["activity_name"]
    position_quota = mitra_data_detail["total"]
    location = mitra_data_detail["location"]
    activity_type = mitra_data_detail["activity_type"]
    logo_mitra = mitra_data_detail["logo"]

    # TODO: EXTRA FIELDS
    start_duration = mitra_data_detail["start_duration"]
    end_duration = mitra_data_detail["end_duration"]
    start_registration = mitra_data_detail["start_registration"]
    end_registration = mitra_data_detail["end_registration"]
    credits_count = mitra_data_detail["credits_count"]
    location_code = mitra_data_detail["location_kotakab_code"]
    mitra_id = mitra_data_detail["mitra_id"]
    certified = mitra_data_detail["certified"]

    #===== Append Data
    id_position_cols.append(id_position)
    position_name_cols.append(position_name)
    mitra_name_cols.append(mitra_name)
    activity_name_cols.append(activity_name)
    position_quota_cols.append(position_quota)
    location_cols.append(location)
    activity_type_cols.append(activity_type)
    logo_mitra_cols.append(logo_mitra)

    # TODO: EXTRA FIELDS
    start_duration_cols.append(start_duration)
    end_duration_cols.append(end_duration)
    start_registration_cols.append(start_registration)
    end_registration_cols.append(end_registration)
    credits_count_cols.append(credits_count)
    location_code_cols.append(location_code)
    mitra_id_cols.append(mitra_id)
    certified_cols.append(certified)

#===== Convert Data to DataFrame
mitra_data_df_full = pd.DataFrame({
    "ID Posisi": id_position_cols,
    "Nama Posisi": position_name_cols,
    "Nama Mitra": mitra_name_cols,
    "Nama Kegiatan": activity_name_cols,
    "Quota Posisi": position_quota_cols,
    "Lokasi Bekerja": location_cols,
    "Tipe Aktivitas": activity_type_cols,
    "URL Logo": logo_mitra_cols,

    # TODO: EXTRA FIELDS
    "Mulai Periode": start_duration_cols,
    "Berakhir Periode": end_duration_cols,
    "Mulai Pendaftaran": start_registration_cols,
    "Berakhir Pendaftaran": end_registration_cols,
    "SKS Diberikan": credits_count_cols,
    "Kode Lokasi": location_code_cols,
    "ID Mitra": mitra_id_cols,
    "Kegiatan Bersertifikat": certified_cols
})


## 2. Process Data COMPACT

In [52]:
url = f'https://api.kampusmerdeka.kemdikbud.go.id/magang/browse/position?offset=0&limit=10000&location_key=&mitra_key=&keyword=&sector_id='
response = requests.get(url, headers=headers)
mitra_data = response.json().get('data')

id_position_cols = []
position_name_cols = []
mitra_name_cols = []
activity_name_cols = []
position_quota_cols = []
location_cols = []
activity_type_cols = []
logo_mitra_cols = []

for mitra_data_detail in mitra_data:
    # fetch data from mitra_data_detail dict in mitra_data list
    id_position = mitra_data_detail["id"]
    position_name = mitra_data_detail["name"]
    mitra_name = mitra_data_detail["mitra_name"]
    activity_name = mitra_data_detail["activity_name"]
    position_quota = mitra_data_detail["total"]
    location = mitra_data_detail["location"]
    activity_type = mitra_data_detail["activity_type"]
    logo_mitra = mitra_data_detail["logo"]

    # Append Data
    id_position_cols.append(id_position)
    position_name_cols.append(position_name)
    mitra_name_cols.append(mitra_name)
    activity_name_cols.append(activity_name)
    position_quota_cols.append(position_quota)
    location_cols.append(location)
    activity_type_cols.append(activity_type)
    logo_mitra_cols.append(logo_mitra)


mitra_data_df_compact = pd.DataFrame({
    "ID Posisi": id_position_cols,
    "Nama Posisi": position_name_cols,
    "Nama Mitra": mitra_name_cols,
    "Nama Kegiatan": activity_name_cols,
    "Quota Posisi": position_quota_cols,
    "Lokasi Bekerja": location_cols,
    "Tipe Aktivitas": activity_type_cols,
    "URL Logo": logo_mitra_cols,
})


In [ ]:
mitra_data_df_full.info()

In [54]:
mitra_data_df_compact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 710 entries, 0 to 709
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID Posisi       710 non-null    object
 1   Nama Posisi     710 non-null    object
 2   Nama Mitra      710 non-null    object
 3   Nama Kegiatan   710 non-null    object
 4   Quota Posisi    710 non-null    int64 
 5   Lokasi Bekerja  710 non-null    object
 6   Tipe Aktivitas  710 non-null    object
 7   URL Logo        710 non-null    object
dtypes: int64(1), object(7)
memory usage: 44.5+ KB


In [56]:
mitra_data_df_compact

,ID Posisi,Nama Posisi,Nama Mitra,Nama Kegiatan,Quota Posisi,Lokasi Bekerja,Tipe Aktivitas,URL Logo
0,6a7bbbab-7508-11ed-b2a2-1a198d2f4c14,2D/3D Animator Intern,BNI,BNI Digital Talent Thematic Internship,13,Kota Jakarta Barat,OFFLINE,https://storage.googleapis.com/kampusmerdeka_k...
1,7659b95a-6429-11ed-b5cc-0a223dcf0f66,Academic Officers Intern,Zenius,Zenius Professionals Practice Program Angkatan 4,6,Kota Jakarta Selatan,BLENDED,https://storage.googleapis.com/kampusmerdeka_k...
2,3f356ec2-6fb0-11ed-a7a1-0278cb7f6574,Accounting Improvement,PT ISS Indonesia,Great ISS Internship Program (GETS Program),2,Kota Tangerang Selatan,OFFLINE,https://storage.googleapis.com/kampusmerdeka_k...
3,99c6bc1f-5c0d-11ed-8806-7a77835150ba,Accounting Intern,Nusantara Infrastructure,[MMN-JTSE] Pengembangan Bisnis Perusahaan mela...,1,Kota Makassar,OFFLINE,https://storage.googleapis.com/kampusmerdeka_k...
4,c53f879e-6a30-11ed-8ebe-a6bfdab34a31,Accounting Officer,PT Bumitama Gunajaya Agro,Officer Development Program,2,Kota Jakarta Selatan,OFFLINE,https://storage.googleapis.com/kampusmerdeka_k...
...,...,...,...,...,...,...,...,...
705,3058ed45-6f9b-11ed-83fd-7e8ddab09462,Workshop Quality & Productivity Control,Bluebird Group,Belajar Birdsama Bluebird,4,"Mampang Prapatan, Jakarta Selatan, DKI Jakarta",OFFLINE,https://storage.googleapis.com/kampusmerdeka_k...
706,947ba8dc-5502-11ed-b209-b250481d78c3,Youth Segment Intern,XL Axiata,DYNAMIC SUSTAINABILITY PROGRAMS (OFFLINE INTER...,4,Kota Jakarta Selatan,OFFLINE,https://storage.googleapis.com/kampusmerdeka_k...
707,67b505b2-6b0c-11ed-8225-d61b0e2692e7,Testing Specialist - Custody & FA System,Bank CIMB Niaga,Custody & Fund Admin System,3,Kota Jakarta Selatan,OFFLINE,https://storage.googleapis.com/kampusmerdeka_k...
708,f52a4546-6b0c-11ed-a22e-c63f61abb566,Treasury Digital Interactive Communication Pr...,Bank CIMB Niaga,Treasury Digital Interactive Communication,3,Kota Jakarta Selatan,OFFLINE,https://storage.googleapis.com/kampusmerdeka_k...


## 3. Output Files [CSV and EXCEL]

In [ ]:
def output_files(df: pd.DataFrame):
    output_types = ["CSV", "Excel"]
    
    print("=== Output File Types ===")
    
    for i in range(len(output_types)):
        format_name = output_types[i]
        print(f'{i+1}. {format_name}')
    
    choose_number = int(input("Pilih format file untuk mengekspor dataframe: ")) - 1

    now = datetime.now()
    # dd-mm-YY H.M
    dt_string = now.strftime("%d-%m-%Y %H.%M")

    if choose_number == 0:
        filename = './' + dt_string + ' - data mitra mbkm.csv'
        df.to_csv(filename, index=False)
    elif choose_number == 1:
        filename = './' + dt_string + ' - data mitra mbkm.xlsx'
        df.to_excel(filename, index=False)
    else:
        raise ValueError("Not a valid input")

    print(f"Creating {output_types[choose_number]} with a file name suffix `- data mitra mbkm`")

output_files(mitra_data_df_full)

## 4. Output Files [JSON]

In [ ]:
def output_files_json(df: pd.DataFrame):
    now = datetime.now()
    # dd-mm-YY H.M
    dt_string = now.strftime("%d-%m-%Y %H.%M")
    filename = './' + dt_string + ' - data mitra mbkm.json'
    df.to_json(filename, orient="records")

output_files_json(mitra_data_df_full)

## Extra: 1. Time Config

In [ ]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()

print("now =", now)

# dd-mm-YY H.M
dt_string = now.strftime("%d-%m-%Y %H.%M")
filename ='./' + dt_string + ' - data mitra mbkm.csv'
print("date and time =", filename)